<a href="https://colab.research.google.com/github/jjome/team1_visual/blob/main/cleansing_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This code is maded to cleansing for landing-club data
 : for team-project in the cource of 8th fintech in SNU

 - The name of train data : "lending_club_2020_train.csv"
 - The name of test data : "lending_club_2020_test.csv"


# Loading data file where is the format


In [ ]:
from google.colab import drive   ## google drive mount
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
## include the libraries
import numpy as np
import pandas as pd
import sys, os

In [ ]:
## loading the csv
dir_name = "/content/drive/MyDrive/Colab Notebooks/통계 실습/"
data_set = "train"
csv = dir_name+"lending_club_2020_train.csv"     # train data set to csv
# csv = dir_name+"lending_club_2020_test.csv"        # test data set to csv
df_ori = pd.read_csv(csv)                          # data frame of csv, i.e. source file
df_ori.head()

<ipython-input-3-ea140d09c95b>:6: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df_ori = pd.read_csv(csv)                          # data frame of csv, i.e. source file


,id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,...,hardship_start_date,hardship_end_date,payment_plan_start_date,hardship_length,hardship_dpd,hardship_loan_status,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount,debt_settlement_flag
0,124989905,6000.0,6000.0,6000.0,36 months,7.97%,187.94,A,A5,Teacher,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N
1,139665484,35000.0,35000.0,35000.0,60 months,14.47%,822.95,C,C2,Product Specialist,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N
2,84253847,23200.0,23200.0,23200.0,60 months,24.99%,680.82,E,E4,Production Foreman,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Y
3,126296074,4500.0,4500.0,4500.0,36 months,9.93%,145.06,B,B2,Office Manager,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N
4,148913312,15000.0,15000.0,15000.0,36 months,16.40%,530.33,C,C4,Secretary,...,May-2020,Jul-2020,May-2020,2.0,0.0,ACTIVE,413.83,11025.25,54.62,N


In [ ]:
Num = len(df_ori)                 # the number of lines to csv.
print(f"The total Number of columns of data are {Num}")

The total Number of columns of data are 1755295


# 전처리 class and function

In [ ]:
dflag = True       ## debuging flag : True make to show the debugging log.
# dflag = False      ## debuging flag : False make to hide the debugging log.

def print_debug(fname:str, msg:str) -> None :
  if dflag == True :
    print(f"[{fname}] : {msg}")
  else :
    pass

In [ ]:
#===============================================================================
# < 전처리 Class >
#
# public method :
#   실행(self)  -> None                        # data의 cleansing을 수행한다.
#   결측치_check (self) -> list                 # feature마다 결측치 숫자를 알려준다.
#   to_csv (self, filename) -> None           # filename을 받아 csv로 저장한다.
#   df_out (self) -> df                       # cleansing이 된 dataframe을 반환한다.
#
# private method :
#   __init__
#   __new_df_정의 (self) :
#   __특수_처리 (self, feat, val) -> None
#   __결측치_처리 (self, feat, val) -> None
#   __이상치_처리 (self, feat, val) -> None
#   __데이터_변형 (self, feat, val) -> None
#===============================================================================
class 전처리 :

  def __init__(self, df_ori, variables) :
    fname = "__init__"

    # Set self variables
    self.item = variables     # all of information for features
    self.df = df_ori          # dataframe to cleansing
    self.feat_list = list(self.item.keys())   # list of features names
    self.del_list = []        # features are dropped in final results.
    print_debug(fname, f"key list : {self.feat_list}")

    # self.df를 input feature들만으로 새로 정의한다.
    self.__new_df_정의()


  ###########################################################
  # 실행 method.
  # 실행 처리 순서 : 각 feature 마다(0~3)
  # 0. 특수 전 처리할 것들을 한다.
  # 1. 결측치를 처리한다.
  # 2. 이상치를 처리한다.
  # 3. 데이터를 변형한다.
  # 4. 데이터를 제거한다.
  ###########################################################
  def 실행(self) -> None :
    fname = "run"

    # x loop start
    for x in self.feat_list :
      item = self.item[x]
      print("="*80)
      print(f"\t[{fname}] ==== Feature name : '{x}' ====")
      print(f"\t[{fname}]\t  결측치 = {item['결측치']}")
      print(f"\t[{fname}]\t  이상치 = {item['이상치']}")
      print(f"\t[{fname}]\t  데이터 변형 = {item['데이터 변형']}")

      if '특수처리' in item :
        self.__특수_처리(x, item)
      self.__결측치_처리(x, item)
      self.__이상치_처리(x, item)
      self.__데이터_변형(x, item)
      print()
    # x loop end

    self.__데이터_제거()


  ###########################################################
  # VARIABLES에 삽입된 변수들만 뽑아 새로운 dataframe을 만듬.
  # __new_df_정의에서 하는 일을 요약하면,
  # 1. 'loan_status'가 결측인 값을 제거
  # 2. 'loan_status'가 Charged Off, Fully Paid인 행들만 추출
  # 3. input feature list를 받아 그 feature들만으로 구성된 새로운 dataframe을 만든다.
  # 4. index를 reset한다.
  ###########################################################
  def __new_df_정의 (self) -> None :
    fname = "__new_df_정의"

    # loan_status가 Null인 값들을 제거
    print(f"\t[{fname}] loan_status가 null인 값 제거 전 data 갯수 : {len(self.df)}")
    missing_indecies = self.df.index[self.df['loan_amnt'].isnull()].tolist()
    self.df = self.df.drop(missing_indecies)
    print(f"\t[{fname}] loan_status가 null인 data 갯수 : {len(missing_indecies)}")
    print(f"\t[{fname}] loan_status가 null인 값 제거 후 data 갯수 : {len(self.df)}")

    # loan_status가 'Charged Off', 'Fully Paid'인 값들만 남긴다.
    print(f"\t[{fname}] Current of loan status 제거 전 data 갯수 : {len(self.df)}")
    self.df = self.df[self.df['loan_status'].isin(['Charged Off', 'Fully Paid'])]
    print(f"\t[{fname}] 'Fully Paid' or 'Charged Off' of loan status 제거 후 data 갯수 : {len(self.df)}")

    # VARIABLES에 있는 변수 들만으로 새로운 dataFrame을 만듬.
    self.df = self.df[self.feat_list]

    # index를 reset한다.
    self.df = self.df.reset_index(drop=True)

    print_debug(fname, self.df.head())


  ###########################################################
  # 결측치 처리 method.
  # - feat : feature name
  # - val : property of feature
  # 처리 방법 :
  # - null_제거 : 값이 없는 행을 제거
  # - null_to_0 : 값이 없는 행을 0 값을 삽입
  # - bootstrap : 값이 없는 행을 값이 있는 다른 행으로 부터 bootstrap하여 값을 삽입
  ###########################################################
  def __결측치_처리 (self, feat, val) -> None :
    fname = "__결측치_처리"
    item = val['결측치']
    flag_drop = True     # True -> 결측치를 처리
    print_debug(fname, f"{feat} : {item} 처리")

    if item == "null_제거" :
      self.df = null_제거_함수(self.df, feat, flag_drop)
    elif item == "null_to_0" :
      self.df = null_to_0_함수(self.df, feat, flag_drop)
    elif item == "bootstrap" :
      self.df = bootstrap_함수(self.df, feat, flag_drop)
    elif item == "없음" :
      pass
    else :
      print_debug(fname, "ERROR : 결측치 처리 방법이  항목에 없음")


  ###########################################################
  # 이상치 처리 method.
  # - feat : feature name
  # - val : property of feature
  # 처리 방법 :
  # - IQR : IQR 값을 제거 하거나 lower(upper)bound로 채움.
  # - IQR_0 : IQR 값을 제거 하거나 0으로 채움.
  # - uad_제거 : 상위, 하위 값을 범위 값을 제거
  ###########################################################
  def __이상치_처리 (self, feat, val) -> None :
    fname = "__이상치_처리"
    item = val['이상치']
    print_debug(fname, f"{feat} : {item} 처리")

    if item == "IQR" :
      item2 = val['IQR_value']
      self.df = IQR_함수(self.df, feat, item2)
    if item == "IQR_0" :
      item2 = val['IQR_value']
      self.df = IQR_0_함수(self.df, feat, item2)
    elif item == "uad_제거" :
      item2 = val['uad_value']
      self.df = uad_제거_함수(self.df, feat, item2)
    elif item == "없음" :
      pass

    else :
      print_debug(fname, "ERROR : 이상치 처리 방법이 항목에 없음")


  ###########################################################
  # 데이터 변형 method.
  # 변형 방법 :
  # - log : logarithm 처리
  # - binary : binary 처리
  # - dummy : dummy화
  # - del : 출력 df에서 지울 del_list에 추가
  ###########################################################
  def __데이터_변형 (self, feat, val) -> None :
    fname = "__데이터_변형"
    item = val['데이터 변형']
    print_debug(fname, f"{feat} : {item} 처리")

    if item == "log" :
      self.df = log_변형_함수(self.df, feat)
    elif item == "binary" :
      self.df = binary_변형_함수(self.df, feat)
    elif item == "dummy" :
      item2 = val['category']
      self.df = dummy_처리_함수(self.df, feat, item2)
    elif item == "del" :
      self.del_list.append(feat)
    elif item == "없음" :
      pass

    else :
      print_debug(fname, "ERROR : 데이터 변형 처리 방법이 항목에 없음")


  ###########################################################
  # 특수 처리 method.
  # 특수 처리 방법 :
  # - max : 두 feature 에서 maximum 값을 넣는다.
  # - avg : 두 feature 의 average 값을 넣는다.
  # - sum : 두 feature 의 sumation 값을 넣는다.
  # - date_to_num : 날짜 형식의 값들에서 'year'과 'month'를 구한다.
  # - zero_to_1 : 0값을 1로 대치한다.
  # - annual_inc : 'annual_inc' feature의 특수 전처리.
  # - total_il_high_credit_limit : 'total_il_high_credit_limit' feature의 특수 전처리.
  ###########################################################
  def __특수_처리 (self, feat, val) -> None :
    fname = "__특수_처리"
    item = val['특수처리']
    print_debug(fname, f"{feat} : {item} 처리")

    if item[0] == "max" :
      self.df = max_val(self.df, feat, item[1])
    elif item[0] == "avg" :
      self.df = avg_val(self.df, feat, item[1])
    elif item[0] == "sum" :
      self.df = sum_val(self.df, feat, item[1])
    elif item[0] == "date_to_num" :
      self.df = date_to_num(self.df, feat)
    elif item[0] == "zero_to_1" :
      self.df = zero_to_1(self.df, feat)
    elif item[0] == "annual_inc" :
      self.df = annual_inc(self.df, feat, item[1])
    elif item[0] == "total_il_high_credit_limit" :
      self.df = total_il_high_credit_limit(self.df, feat)
    elif item == "없음" :
      pass
    else :
      print_debug(fname, "ERROR : 데이터 변형 처리 방법이 항목에 없음")


  ###########################################################
  # 데이터 제거 method.
  # 데이터 제거 방법 :
  ###########################################################
  def __데이터_제거 (self) -> None :
    fname = "__데이터_제거"

    # 추가로 제거할 feature 리스트를 추가한다.
    add_del_list = ['year','month','mths_since_recent_bc_dlq_nan', 'mths_since_recent_inq_nan']
    self.del_list = self.del_list + add_del_list
    print(f"\t{self.del_list} feature 제거")

    # df에서 del_list에 적힌 feaure들을 제거한다.
    self.df.drop(columns=self.del_list, inplace=True)
    self.df.describe(include='all')
    print(self.df.head())



  ###########################################################
  # 결측치 check method.
  # check 항목 :
  # - null_값 : 값이 없는 행을 check
  # - nan_값 : nan 값을 가진 행을 check
  # [xxx:fix me] : 결측히 체크 리스트를 반환
  ###########################################################
  def 결측치_check (self) :
    fname = "결측치_check"

    for x in self.feat_list :
      flag_drop = False
      item = self.item[x]
      print_debug(fname, f"{x} : {item['결측치']} 처리")

      # Null 값 결측치 check
      df_out = null_제거_함수(self.df, x, flag_drop)


  ###########################################################
  # Save csv from self.df method.
  # - filename : name of file to out
  ###########################################################
  def to_csv (self, filename:str) -> None :
    fname = "to_csv"

    print_debug(fname, f"Save the csv in {filename}")
    self.df.to_csv(filename, index=False, encoding='utf-8-sig')


  ###########################################################
  # df_out method.
  ###########################################################
  def df_out (self) :
    fname = "df_out"

    return self.df

## 결측치 처리 함수들

### bootstrap 적용

In [ ]:
###########################################################
# bootstrap 함수
# - df : data Frame
# - feat : feature name
# - flag_drop : True 일때 결측값들을 드롭한다.
###########################################################
def bootstrap_함수 (df, feat:str, flag_drop:bool) :
  fname = "bootstrap_함수"

  notnull_values = df[feat].dropna()  # 결측치가 아닌 값들

  if feat == 'mths_since_recent_revol_delinq' or feat == 'mths_since_recent_inq' :
    print_debug(fname, f"null을 bootstrap으로 처리 실행 전 data 개수 : {len(df)}")

    np.random.seed(1004) ## random seed 추가
    bootstrap_samples = np.random.choice(notnull_values, size=len(df[feat]) - notnull_values.isnull().sum(), replace=True)  ## 부트스트랩 샘플 생성

    tmp = df[feat].copy()
    tmp[tmp.isnull()] = bootstrap_samples
    df[feat] = tmp  # 결측치에 부트스트랩 샘플 채워넣기

    print_debug(fname, f"bootstrap으로 처리 실행 된 data 개수 : {len(bootstrap_samples)}")
    print_debug(fname, f"null을 bootstrap으로 처리 실행 후 data 개수 : {len(df)}")

    # 'mths_since_recent_inq' feature에 대한 특수 처리
    if feat == 'mths_since_recent_inq' :
      df[feat].loc[df[feat] == 0] = 1 # 0값을 1로 변경
    df[feat].fillna(0, inplace=True)

  else :
    # flag_drop = True  : drop 하여 저장하기
    #           = False : check용 출력하기
    if flag_drop :
      np.random.seed(1004) ## random seed 추가
      bootstrap_samples = np.random.choice(notnull_values, size=len(df[feat]) - notnull_values.isnull().sum(), replace=True)  # 부트스트랩 샘플 생성

      tmp = df[feat].copy()
      tmp[tmp.isnull()] = bootstrap_samples
      df[feat] = tmp  # 결측치에 부트스트랩 샘플 채워넣기
      print_debug(fname, f"bootstrap으로 처리 실행 된 data 개수 : {len(bootstrap_samples)}")
      print_debug(fname, f"null을 bootstrap으로 처리 실행 후 data 개수 : {len(df)}")

    else :
      df = df.loc[notnull_values]
      print_debug(fname, f"The number of null data of '{feat}' is {len(df)}")

  return df

### null 제거


In [ ]:
###########################################################
# null 제거 함수
# - df : data Frame
# - feat : feature name
# - flag_drop : True 일때 결측값들을 드롭한다.
###########################################################
def null_제거_함수 (df, feat:str, flag_drop:bool) :
  fname = "null_제거_함수"

  # 결측치 필터링
  missing_indecies = df.index[df[feat].isna()].tolist()

  # flag_drop = True  : drop 하여 저장하기
  #           = False : check용 출력하기
  if flag_drop :
    print_debug(fname, f"null 제거 실행 전 data 개수 : {len(df)}")
    new_df = df.drop(missing_indecies)
    print_debug(fname, f"null 제거 data 개수 : {len(missing_indecies)}")
    print_debug(fname, f"null 제거 실행 후 data 개수 : {len(new_df)}")

  else :
    new_df = df.loc[missing_indecies]
    print_debug(fname, f"The number of null data of '{feat}' is {len(new_df)}")

  return new_df

### null to 0 처리

In [ ]:
###########################################################
# null to 0 처리 함수
# - df : data Frame
# - feat : feature name
###########################################################
def null_to_0_함수 (df, feat:str, flag_drop:bool) :
  fname = "null_to_0_함수"

  # 결측치 필터링
  missing_indecies = df.index[df[feat].isnull()].tolist()

  # flag_drop = True  : drop 하여 저장하기
  #           = False : check용 출력하기
  if flag_drop :
    print_debug(fname, f"null을 0으로 치환 실행 전 data 개수 : {len(df)}")
    df.loc[missing_indecies, feat] = 0    ## 결측치를 0으로 대치
    print_debug(fname, f"null data 개수 : {len(missing_indecies)}")
    print_debug(fname, f"null을 0으로 치환  실행 후 data 개수 : {len(df)}")

  else :
    df = df.loc[missing_indecies]
    print_debug(fname, f"The number of null data of '{feat}' is {len(df)}")

  return df

## 이상치 처리 함수들

### IQR_함수

In [ ]:
###########################################################
# IQR 함수
# - df : data Frame
# - feat : feature name
# - val : there are a parameter and drop_flag
###########################################################
def IQR_함수 (df, feat, val:list) :
  fname = "IQR_함수"
  IQR_value = val[0]
  drop_flag = val[1]

  # IQR 값 계산
  Q1 = df[feat].quantile(0.25)
  Q3 = df[feat].quantile(0.75)
  IQR = Q3 - Q1
  print_debug(fname, f"Q1 value = {Q1} ")
  print_debug(fname, f"Q3 value = {Q3} ")
  print_debug(fname, f"IQR value = {IQR} ")

  # IQR 값으로 부터 lower bound, upper bound를 구함
  lower_bound = Q1 - IQR_value * IQR
  upper_bound = Q3 + IQR_value * IQR
  print_debug(fname, f"lower bound value = {lower_bound} ")
  print_debug(fname, f"upper bound value = {upper_bound} ")

  # 이상치 필터링
  outliers = df.index[(df[feat] < lower_bound) | (df[feat] > upper_bound)].tolist()

  if drop_flag :
    # 이상치를 제거한다.
    print(f"\t[{fname}] IQR 실행 전 data 개수 {drop_flag} : {len(df)}")
    df.drop(outliers, axis=0, inplace=True)
    print(f"\t[{fname}] IQR 이상치 data 개수 {drop_flag} : {len(outliers)}")
    print(f"\t[{fname}] IQR 실행 후 data 개수 {drop_flag} : {len(df)}")

  else :
    # 이상치를 lowerbound or upperbound로 대치한다.
    print(f"\t[{fname}] IQR 실행 전 data 개수 {drop_flag} : {len(df)}")
    df.loc[df[feat] < lower_bound, feat] = lower_bound
    df.loc[df[feat] > upper_bound, feat] = upper_bound
    print(f"\t[{fname}] IQR 이상치 data 개수 {drop_flag} : {len(outliers)}")
    print(f"\t[{fname}] IQR 실행 후 data 개수 {drop_flag} : {len(df)}")

  return df

### IQR_0_함수

In [ ]:
###########################################################
# IQR_0 함수
# - df : data Frame
# - feat : feature name
# - IQR_value : the parameter for IQR method
# - val : there are a parameter and drop_flag
###########################################################
def IQR_0_함수 (df, feat, val:list) :
  fname = "IQR_0_함수"
  IQR_value = val[0]
  drop_flag = val[1]

  # IQR 값 계산
  Q1 = df[feat].quantile(0.25)
  Q3 = df[feat].quantile(0.75)
  IQR = Q3 - Q1
  print_debug(fname, f"Q1 value = {Q1} ")
  print_debug(fname, f"Q3 value = {Q3} ")
  print_debug(fname, f"IQR value = {IQR} ")

  # IQR 값으로 부터 lower bound, upper bound를 구함
  lower_bound = Q1 - IQR_value * IQR
  upper_bound = Q3 + IQR_value * IQR
  print_debug(fname, f"lower bound value = {lower_bound} ")
  print_debug(fname, f"upper bound value = {upper_bound} ")

  # 이상치 필터링
  outliers = df.index[(df[feat] < lower_bound) | (df[feat] > upper_bound)].tolist()

  if drop_flag :
    # 이상치를 제거한다.
    print(f"\t[{fname}] IQR 실행 전 data 개수 {drop_flag} : {len(df)}")
    df.drop(outliers, axis=0, inplace=True)
    print(f"\t[{fname}] IQR 이상치 data 개수 {drop_flag} : {len(outliers)}")
    print(f"\t[{fname}] IQR 실행 후 data 개수 {drop_flag} : {len(df)}")

  else :
    # 이상치를 0으로 대치한다.
    print(f"\t[{fname}] IQR 실행 전 data 개수 {drop_flag} : {len(df)}")
    df.loc[df[feat] < lower_bound, feat] = 0
    df.loc[df[feat] > upper_bound, feat] = 0
    print(f"\t[{fname}] IQR 이상치 data 개수 {drop_flag} : {len(outliers)}")
    print(f"\t[{fname}] IQR 실행 후 data 개수 {drop_flag} : {len(df)}")

  return df

### uad_제거_함수

In [ ]:
###########################################################
# uad 제거 함수
# - df : data Frame
# - feat : feature name
# - uad_value : the parameters of upper-bound and lower-bound
###########################################################
def uad_제거_함수 (df, feat:str, uad_value:tuple) :
  fname = "uad_제거_함수"

  # lowerbound와 upperbound값음 구함
  lowerbound = df[feat].quantile(uad_value[0])
  upperbound = df[feat].quantile(1.0 - uad_value[1])

  # 이상치에 해당되는 값들
  outliers = df[(df[feat] < lowerbound) | (df[feat] > upperbound)]

  # 새로운 df에 범위에 해당되는 값들을 넣음
  new_df = df[(df[feat] >= lowerbound) & (df[feat] <= upperbound) ]

  print(f"\t[{fname}] uad 제거 실행 전 data 개수 : {len(df)}")
  print(f"\t[{fname}] {uad_value[0]}% 미만 and {1-uad_value[1]}% 초과 data 개수 : {len(outliers)}")
  print(f"\t[{fname}] uad 제거 실행 후 data 개수 : {len(new_df)}")

  return new_df

## 데이터 변형 함수들

### log_변형_함수

In [ ]:
###########################################################
# Log 변형 함수
# numpy의 log1p를 사용함 : x' = log(1+x)
# - df : data Frame
# - feat : feature name
###########################################################
def log_변형_함수 (df, feat) :
  fname = "log_변형_함수"

  # log 값 계산
  df[feat] = np.log1p(df[feat])

  return df

### binary_변형_함수

In [ ]:
###########################################################
# Binary 변형 함수
# numpy의 log1p를 사용함 : f(x) = log(1+x)
# - df : data Frame
# - feat : feature name
###########################################################
def binary_변형_함수 (df, feat) :
  fname = "binary_변형_함수"

  # Binary 변형
  df[feat] = np.digitize(df[feat], bins=[0.1])

  return df

### dummy_처리_함수

In [ ]:
###########################################################
# Binary 변형 함수
# numpy의 log1p를 사용함 : f(x) = log(1+x)
# - df : data Frame
# - feat : feature name
# - val[0] : category화 True or False
# - val[1] : category화 dummy_range
# - val[2] : category화 dummy_labels
###########################################################
def dummy_처리_함수 (df, feat:str, val:tuple) :
  fname = "dummy_처리_함수"
  category = val[0]

  # dummy_처리
  if category :   # category=True
    dummy_range = val[1]
    dummy_labels = val[2]

    df['tmp'] = pd.cut(df[feat], bins=dummy_range, labels=dummy_labels, right=False)

    # dummy 변수 생성
    tmp_dummies = pd.DataFrame()
    tmp_dummies = pd.get_dummies(df['tmp'], prefix=feat)

    # dummy 변수를 원본 데이터프레임에 합치기
    df = pd.concat([df, tmp_dummies], axis=1)
    df.drop(feat, axis=1, inplace=True)
    df.drop('tmp', axis=1, inplace=True)

  else :
    if feat == 'emp_length' :
      # dummy 변수 생성
      dummies = pd.get_dummies(df[feat])
      tmp_dummies = pd.DataFrame()

      tmp_dummies['emp_length_emp_length_less_3'] = dummies['< 1 year'].astype(int) \
                          + dummies['1 year'].astype(int) \
                          + dummies['2 years'].astype(int) \
                          + dummies['3 years'].astype(int)

      tmp_dummies['emp_length_emp_length_3_to_7'] = dummies['4 years'].astype(int) \
                          + dummies['5 years'].astype(int) \
                          + dummies['6 years'].astype(int) \
                          + dummies['7 years'].astype(int) \
                          + dummies['8 years'].astype(int) \
                          + dummies['9 years'].astype(int)

      tmp_dummies['emp_length_emp_length_10+'] = dummies['10+ years'].astype(int)

      # dummy 변수를 원본 데이터프레임에 합치기
      df = pd.concat([df, tmp_dummies], axis=1)
      df.drop(feat, axis=1, inplace=True)

    else :
      dummies = pd.get_dummies(df[feat], prefix=feat)

      # dummy 변수를 원본 데이터프레임에 합치기
      df = pd.concat([df, dummies], axis=1)
      df.drop(feat, axis=1, inplace=True)

  return df

## 특수 전 처리 함수들

### max_val

In [ ]:
###########################################################
# max_val 함수
# - df : data Frame
# - feat : feature name
# - feat2 : feature name
###########################################################
def max_val (df, feat:str, feat2:str) :
  fname = "max_val"
  print_debug(fname, f"To obtain the max value of {feat} and {feat2}")

  if feat == 'revol_util' :
    df[feat] = df[feat].str.rstrip('%').astype(float)

  # feat가 결측이면 jfeat2 값으로 대치
  df[feat] = df[feat].fillna(df[feat2])

  # a, b 사이 최대값을 계산하고, b가 결측치이면 a값을 그대로 사용
  max_A_B = df[[feat, feat2]].max(axis=1)
  max_A_B[df[feat2].isnull()] = df[feat]
  df[feat] = max_A_B

  return df

### avg_val

In [ ]:
###########################################################
# avg_val 함수
# - df : data Frame
# - feat : feature name
# - feat2 : feature name
###########################################################
def avg_val (df, feat:str, feat2:str) :
  fname = "max_val"
  print_debug(fname, f"To obtain the average value of {feat} and {feat2}")

  # a, b 사이 평균값을 계산하고, b가 결측치이면 a값을 그대로 사용
  mean_A_B = df[[feat, feat2]].mean(axis=1)
  mean_A_B[df[feat2].isnull()] = df[feat]
  df[feat] = mean_A_B

  return df

### sum_val

In [ ]:
###########################################################
# sum_val 함수
# - df : data Frame
# - feat : feature name
# - feat2 : feature name
###########################################################
def sum_val (df, feat:str, feat2:str) :
  fname = "sum_val"
  print_debug(fname, f"To obtain the sum value of {feat} and {feat2}")

  # a, b 사이 합을 계산한다.
  df[feat] = df[feat] + df[feat2]

  return df

### date_to_num

In [ ]:
###########################################################
# date_to_num 함수
# - df : data Frame
# - feat : feature name
###########################################################
def date_to_num (df, feat) :
  fname = "date_to_num"

  # 날짜형 변수의 경우, 'year'과 'month' feature로 나눠서 저장.
  df[feat] = pd.to_datetime(df[feat], format='%b-%Y')
  df['year'] = df[feat].dt.year.astype(float)
  df['month'] = df[feat].dt.month.astype(float)

  return df

### zero_to_1

In [ ]:
###########################################################
# zero_to_1 함수
# - df : data Frame
# - feat : feature name
###########################################################
def zero_to_1 (df, feat) :
  fname = "zero_to_1"
  # 0인 값들을 1로 대치
  df[feat].loc[df[feat] == 0] = 1

  return df

### annual_inc

In [ ]:
###########################################################
# annual_inc 함수
# - df : data Frame
# - feat : feature name
###########################################################
def annual_inc (df, feat, feat2) :
  fname = "annual_inc"
  df.dropna(subset=[feat], inplace=True) # 결측치 제거(행삭제)

  # annual_inc이 1보다 작은 경우 annual_inc_joint 값으로 변경
  df[feat] = df[feat2].where(df[feat] <= 1, df[feat])

  return df

### total_il_high_credit_limit

In [ ]:
###########################################################
# total_il_high_credit_limit 함수
# - df : data Frame
# - feat : feature name
###########################################################
def total_il_high_credit_limit (df, feat) :
  fname = "total_il_high_credit_limit"

  df[feat] = pd.cut(
    df[feat],
    bins=[0, 3525, 15500, 24642, 33923, 45142, 61000, 88913, 9999999999],  # 모두 정수로 지정
    labels=[
        'q1',
        'q2',
        'q3',
        'q4',
        'q5',
        'q6',
        'q7',
        'q8']
  )

  return df

# feature 속성 dictionary

In [ ]:
###########################################################
# VARIABLES : feature의 전처리 정보 dictionary
# 구현된 method
# - 결측치 =
#     'null_제거'
#     'null_to_0'
#     'bootstrap'
#     '없음'
#
# - 이상치 =
#     'IQR'
#     'uad_제거'
#     '없음'
# - 'IQR_value' = [factor=1.5, drop_flag]   # drap_flag = True 일때는 이상치를 버림
#                                           # drap_flag = False 일때는 lower(upper)bound 값으로 대치
# - 'uad_value' = [lowerbound, upperbount]
#
# - 데이터 변형 =
#     'log'
#     'binary'
#     'dummy'
#     '없음'
# - category = [category_flag, dummy_range, dummy_label]     # category_flag = True 일때 뒤의 dummy_range와 dummy_label을 받아 범주형태로 더비 변수를 만든다.
#                                                            # category_flag = False 일때 feature의 각 항목별로 더미화 한다.
#
# - 특수 처리 = [val, option]
#     ['max', second feature]
#     ['sum', second feature]
#     ['avg', second feature]
#     ['date_to_num']
#     ['annual_inc', 'annual_inc_joint']
#     ['total_il_high_credit_limit]
###########################################################

VARIABLES = {
    'id' : {      # 1: done
        '결측치' : '없음',
        '이상치' : '없음',
        '데이터 변형' : '없음'
    }
    , 'loan_status' : {        # 3: done
        '결측치' : '없음',
        '이상치' : '없음',
        '데이터 변형' : 'dummy',
        'category' : [False]
    }
    , 'acc_now_delinq' : {      # 1: done
        '결측치' : 'null_제거',
        '이상치' : '없음',
        '데이터 변형' : 'binary'
    }
    , 'addr_state' : {        # 3: done
        '결측치' : 'null_제거',
        '이상치' : '없음',
        '데이터 변형' : 'dummy',
        'category' : [False]
    }
    , 'annual_inc' : {        # 5: done     # annual_inc이 1보다 작은 경우 annual_inc_joint 값으로 변경
        '특수처리' : ['annual_inc', 'annual_inc_joint'],
        '결측치' : 'null_제거',
        '이상치' : '없음',
        '데이터 변형' : 'log'
    }
    , 'annual_inc_joint' : {        # 5: done     # annual_inc이 1보다 작은 경우 annual_inc_joint 값으로 변경
        '결측치' : '없음',
        '이상치' : '없음',
        '데이터 변형' : 'del'
    }
    , 'application_type' : {   # 7: done
        '결측치' : 'null_제거',
        '이상치' : '없음',
        '데이터 변형' : 'dummy',
        'category' : [False]
    }
    , 'sec_app_chargeoff_within_12_mths' : {    # 11: done   # 공동대출자와 합치기
        '결측치' : 'null_to_0',
        '이상치' : '없음',
        '데이터 변형' : 'del'
    }
    , 'chargeoff_within_12_mths' : {    # 11: done   # 공동대출자와 합치기
        '특수처리' : ['sum', 'sec_app_chargeoff_within_12_mths'],
        '결측치' : 'null_제거',
        '이상치' : '없음',
        '데이터 변형' : 'binary'
    }
    , 'sec_app_collections_12_mths_ex_med' : {  # 13: done   # 공동대출자와 합치기
        '결측치' : 'null_to_0',
        '이상치' : '없음',
        '데이터 변형' : 'del'
    }
    , 'collections_12_mths_ex_med' : {  # 13: done   # 공동대출자와 합치기
        '특수처리' : ['sum', 'sec_app_collections_12_mths_ex_med'],
        '결측치' : 'null_제거',
        '이상치' : '없음',
        '데이터 변형' : 'binary'
    }
    , 'delinq_2yrs' : {                 # 14 : done
        '결측치' : 'null_제거',
        '이상치' : '없음',
        '데이터 변형' : 'binary'
    }
    , 'delinq_amnt' : {                 # 15 : done
        '결측치' : 'null_제거',
        '이상치' : '없음',
        '데이터 변형' : 'binary'
    }
    , 'dti_joint' : {                     # 18 : done
        '결측치' : '없음',
        '이상치' : '없음',
        '데이터 변형' : 'del'
    }
    , 'dti' : {                           # 17 : done
        '특수처리' : ['max', 'dti_joint'],
        '결측치' : 'null_제거',
        '이상치' : 'IQR',
        'IQR_value' : [1.5, False],
        '데이터 변형' : '없음'
    }
    , 'emp_length' : {
        '결측치' : 'bootstrap',           # 20 : done
        '이상치' : '없음',
        '데이터 변형' : 'dummy',
        'category' : [False]
    }
    , 'sec_app_fico_range_high' : {     # 23 : done
        '결측치' : 'null_to_0',
        '이상치' : '없음',
        '데이터 변형' : 'del'
    }
    , 'fico_range_high' : {             # 22 : done
        '특수처리' : ['max', 'sec_app_fico_range_high'],
        '결측치' : 'null_to_0',
        '이상치' : '없음',
        '데이터 변형' : '없음'
    }
    , 'home_ownership' : {              # 23 : done
        '결측치' : 'null_제거',
        '이상치' : '없음',
        '데이터 변형' : 'dummy',
        'category' : [False]
    }
    , 'inq_last_6mths' : {              # 33 : done
        '결측치' : 'null_제거',
        '이상치' : '없음',
        '데이터 변형' : 'binary'
    }
    , 'issue_d' : {                     # 36 : done   # issue_d를 연도로 변환
        '특수처리' : ['date_to_num'],
        '결측치' : '없음',
        '이상치' : '없음',
        '데이터 변형' : 'del'
    }
    , 'loan_amnt' : {                   # 42 : done
        '결측치' : 'null_제거',
        '이상치' : '없음',
        '데이터 변형' : '없음',
    }
    , 'mths_since_last_delinq' : {      # 51 : done      # 0값을 1로 바꾸기
        '특수처리' : ['zero_to_1'],
        '결측치' : 'null_to_0',
        '이상치' : '없음',
        '데이터 변형' : 'binary'
    }
    , 'mths_since_last_major_derog' : { # 52 : done    # 이상값을 0으로 바꿉니다.
        '결측치' : '없음',
        '이상치' : 'IQR_0',
        'IQR_value' : [1.5, False],
        '데이터 변형' : 'binary'
    }
    , 'mths_since_last_record' : {      # 53 : done
        '결측치' : 'null_to_0',
        '이상치' : '없음',
        '데이터 변형' : 'binary'
    }
    , 'mths_since_recent_bc' : {        # 55 : done
        '결측치' : 'bootstrap',
        '이상치' : '없음',
        '데이터 변형' : 'log'
    }
    , 'mths_since_recent_bc_dlq' : {    # 56 : done       # 0값을 1로 변경  #결측값을 0으로 변경
        '특수처리' : ['zero_to_1'],
        '결측치' : 'null_to_0',
        '이상치' : '없음',
        '데이터 변형' : 'dummy',
        'category' : [True, (-1, 1, 21, 36, 56, 1000000000000000), ('nan', 'q1', 'q2', 'q3', 'q4')]
    }
    , 'mths_since_recent_inq' : {      # 57 : done       # issue_d가 2011 이하이면 부트스트랩 적용  # 0값을 1로 변경  #결측값을 0으로 변경
        '결측치' : 'bootstrap',
        '이상치' : '없음',
        '데이터 변형' : 'dummy',
        'category' : [True, (-1, 1, 3, 6, 12, 24, 1000000000000000), ('nan', 'q1', 'q2', 'q3', 'q4', 'q5')]
    }
    , 'mths_since_recent_revol_delinq' : {  # 58 : not yet     # issue_d가 2011 이하이면 부트스트랩 적용
        '결측치' : 'bootstrap',
        '이상치' : '없음',
        '데이터 변형' : 'binary',
    }
    , 'sec_app_num_rev_accts' : {                 # 62 : done    # 변수 합성 with 'sec_app_num_rev_accts'
        '결측치' : '없음',
        '이상치' : '없음',
        '데이터 변형' : 'del'
    }
    , 'num_actv_rev_tl' : {                 # 62 : done    # 변수 합성 with 'sec_app_num_rev_accts'
        '결측치' : 'bootstrap',
        '이상치' : 'IQR',
        'IQR_value' : [1.5, False],
        '데이터 변형' : '없음'
    }
    , 'num_rev_accts' : {                   # 67 : done
        '결측치' : 'bootstrap',
        '이상치' : '없음',
        '데이터 변형' : 'log'
    }
    , 'num_rev_tl_bal_gt_0' : {             # 68 : done
        '결측치' : 'bootstrap',
        '이상치' : '없음',
        '데이터 변형' : 'binary'
    }
    , 'num_tl_120dpd_2m' : {                # 70 : done
        '결측치' : 'bootstrap',
        '이상치' : '없음',
        '데이터 변형' : 'binary'
    }
    , 'num_tl_30dpd' : {                    # 71 : done
        '결측치' : 'bootstrap',
        '이상치' : '없음',
        '데이터 변형' : 'binary'
    }
    , 'open_acc' : {                    # 71 : done
        '결측치' : 'null_제거',
        '이상치' : 'IQR',
        'IQR_value' : [1.5, False],
        '데이터 변형' : '없음'
    }
    , 'pct_tl_nvr_dlq' : {                  # 83 : done
        '결측치' : 'bootstrap',
        '이상치' : '없음',
        '데이터 변형' : '없음'
    }
    , 'percent_bc_gt_75' : {                # 84 : done
        '결측치' : 'bootstrap',
        '이상치' : '없음',
        '데이터 변형' : '없음'
    }
    , 'pub_rec' : {                         # 86 : done
        '결측치' : 'null_제거',
        '이상치' : '없음',
        '데이터 변형' : 'binary'
    }
    , 'pub_rec_bankruptcies' : {            # 87 : done
        '결측치' : 'bootstrap',
        '이상치' : '없음',
        '데이터 변형' : 'binary'
    }
    , 'purpose' : {                         # 88 : done
        '결측치' : 'null_제거',
        '이상치' : '없음',
        '데이터 변형' : 'dummy',
        'category' : [False]
    }
    , 'revol_bal_joint' : {                   # 91 : done
        '결측치' : '없음',
        '이상치' : '없음',
        '데이터 변형' : 'del'
    }
    , 'revol_bal' : {                         # 91 : done
        '특수처리' : ['avg', 'revol_bal_joint'],
        '결측치' : 'null_제거',
        '이상치' : '없음',
        '데이터 변형' : 'log'
    }
    , 'sec_app_revol_util' : {                # 92 : done
        '결측치' : '없음',
        '이상치' : '없음',
        '데이터 변형' : 'del'
    }
    , 'revol_util' : {                      # 92 : done
        '특수처리' : ['max', 'sec_app_revol_util'],
        '결측치' : 'bootstrap',
        '이상치' : '없음',
        '데이터 변형' : '없음'
    }
    , 'tax_liens' : {                       # 94 : done
        '결측치' : 'null_제거',
        '이상치' : '없음',
        '데이터 변형' : 'binary'
    }
    , 'term' : {                            # 95 : done
        '결측치' : 'null_제거',
        '이상치' : '없음',
        '데이터 변형' : 'dummy',
        'category' : [False]
    }
    , 'total_acc' : {                       # 100 : done
        '결측치' : 'null_제거',
        '이상치' : '없음',
        '데이터 변형' : 'log'
    }
    , 'total_bal_ex_mort' : {               # 101 : done
        '결측치' : 'bootstrap',
        '이상치' : '없음',
        '데이터 변형' : 'log'
    }
    , 'total_bc_limit' : {                  # 103 : done
        '결측치' : 'bootstrap',
        '이상치' : '없음',
        '데이터 변형' : 'log'
    }
    , 'total_il_high_credit_limit' : {        # 105 : done    # 타입변경
        '특수처리' : ['total_il_high_credit_limit'],
        '결측치' : 'bootstrap',
        '이상치' : '없음',
        '데이터 변형' : 'dummy',
        'category' : [False]
    }
}

print(len(VARIABLES))

51


# 실행 블럭

In [ ]:
dflag = False   ## debuging flag : True make the log for debug.
tmp = 전처리(df_ori, VARIABLES)

	[__new_df_정의] loan_status가 null인 값 제거 전 data 갯수 : 1755295
	[__new_df_정의] loan_status가 null인 data 갯수 : 1
	[__new_df_정의] loan_status가 null인 값 제거 후 data 갯수 : 1755294
	[__new_df_정의] Current of loan status 제거 전 data 갯수 : 1755294
	[__new_df_정의] 'Fully Paid' or 'Charged Off' of loan status 제거 후 data 갯수 : 1115888


In [ ]:
dflag = False   ## debuging flag : True make the log for debug.
# tmp.결측치_check()

In [ ]:
dflag = False   ## debuging flag : True make the log for debug.
tmp.실행()

	[run] ==== Feature name : 'id' ====
	[run]	  결측치 = 없음
	[run]	  이상치 = 없음
	[run]	  데이터 변형 = 없음

	[run] ==== Feature name : 'loan_status' ====
	[run]	  결측치 = 없음
	[run]	  이상치 = 없음
	[run]	  데이터 변형 = dummy

	[run] ==== Feature name : 'acc_now_delinq' ====
	[run]	  결측치 = null_제거
	[run]	  이상치 = 없음
	[run]	  데이터 변형 = binary

	[run] ==== Feature name : 'addr_state' ====
	[run]	  결측치 = null_제거
	[run]	  이상치 = 없음
	[run]	  데이터 변형 = dummy

	[run] ==== Feature name : 'annual_inc' ====
	[run]	  결측치 = null_제거
	[run]	  이상치 = 없음
	[run]	  데이터 변형 = log

	[run] ==== Feature name : 'annual_inc_joint' ====
	[run]	  결측치 = 없음
	[run]	  이상치 = 없음
	[run]	  데이터 변형 = del

	[run] ==== Feature name : 'application_type' ====
	[run]	  결측치 = null_제거
	[run]	  이상치 = 없음
	[run]	  데이터 변형 = dummy

	[run] ==== Feature name : 'sec_app_chargeoff_within_12_mths' ====
	[run]	  결측치 = null_to_0
	[run]	  이상치 = 없음
	[run]	  데이터 변형 = del

	[run] ==== Feature name : 'chargeoff_within_12_mths' ====
	[run]	  결측치 = null_제거
	[run]	  이상치 = 없음
	[

<ipython-input-20-3c8b656ce1cc>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[feat].loc[df[feat] == 0] = 1



	[run] ==== Feature name : 'mths_since_last_major_derog' ====
	[run]	  결측치 = 없음
	[run]	  이상치 = IQR_0
	[run]	  데이터 변형 = binary
	[IQR_0_함수] IQR 실행 전 data 개수 False : 1115850
	[IQR_0_함수] IQR 이상치 data 개수 False : 371
	[IQR_0_함수] IQR 실행 후 data 개수 False : 1115850

	[run] ==== Feature name : 'mths_since_last_record' ====
	[run]	  결측치 = null_to_0
	[run]	  이상치 = 없음
	[run]	  데이터 변형 = binary

	[run] ==== Feature name : 'mths_since_recent_bc' ====
	[run]	  결측치 = bootstrap
	[run]	  이상치 = 없음
	[run]	  데이터 변형 = log

	[run] ==== Feature name : 'mths_since_recent_bc_dlq' ====
	[run]	  결측치 = null_to_0
	[run]	  이상치 = 없음
	[run]	  데이터 변형 = dummy


<ipython-input-20-3c8b656ce1cc>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[feat].loc[df[feat] == 0] = 1



	[run] ==== Feature name : 'mths_since_recent_inq' ====
	[run]	  결측치 = bootstrap
	[run]	  이상치 = 없음
	[run]	  데이터 변형 = dummy


<ipython-input-7-ecac98e87fa5>:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[feat].loc[df[feat] == 0] = 1 # 0값을 1로 변경



	[run] ==== Feature name : 'mths_since_recent_revol_delinq' ====
	[run]	  결측치 = bootstrap
	[run]	  이상치 = 없음
	[run]	  데이터 변형 = binary

	[run] ==== Feature name : 'sec_app_num_rev_accts' ====
	[run]	  결측치 = 없음
	[run]	  이상치 = 없음
	[run]	  데이터 변형 = del

	[run] ==== Feature name : 'num_actv_rev_tl' ====
	[run]	  결측치 = bootstrap
	[run]	  이상치 = IQR
	[run]	  데이터 변형 = 없음
	[IQR_함수] IQR 실행 전 data 개수 False : 1115850
	[IQR_함수] IQR 이상치 data 개수 False : 31966
	[IQR_함수] IQR 실행 후 data 개수 False : 1115850

	[run] ==== Feature name : 'num_rev_accts' ====
	[run]	  결측치 = bootstrap
	[run]	  이상치 = 없음
	[run]	  데이터 변형 = log

	[run] ==== Feature name : 'num_rev_tl_bal_gt_0' ====
	[run]	  결측치 = bootstrap
	[run]	  이상치 = 없음
	[run]	  데이터 변형 = binary

	[run] ==== Feature name : 'num_tl_120dpd_2m' ====
	[run]	  결측치 = bootstrap
	[run]	  이상치 = 없음
	[run]	  데이터 변형 = binary

	[run] ==== Feature name : 'num_tl_30dpd' ====
	[run]	  결측치 = bootstrap
	[run]	  이상치 = 없음
	[run]	  데이터 변형 = binary

	[run] ==== Feature name : 'open_ac

In [ ]:
from datetime import date

dflag = False   ## debuging flag : True make the log for debug.

# 날짜 추출
today = date.today()
date = today.strftime("%Y%m%d")
print(f"Today is {date}")

# Save filename
filename = dir_name+"lc_2020_train_ppd_"+str(date)+".csv"  # train data set to csv
# filename = dir_name+"lc_2020_test_ppd_"+str(date)+".csv"  # train data set to csv

# Save to csv
tmp.to_csv(filename)

Today is 20240429
